## Video MCTF coding

In [1]:
from IPython.display import Video

In [ ]:
#!pip install -r ../requirements.txt

### Help about basic functionality

In [ ]:
!python3 ../src/MCTF.py -h

### Help to encode

In [ ]:
!python3 ../src/MCTF.py encode -h

### Encode and decoding a remote video

In [6]:
Video("http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4")

In [ ]:
# Use -o for original video, --num_gops to specify how many GOPs to encode
# Total frames = gop_size * num_gops (16 * 2 = 32 frames)
!python3 ../src/MCTF.py encode -o http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4 --gop_size 16 --num_gops 2

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
# Video is automatically created at /tmp/decoded.mp4
Video("../tmp/encoded.mp4")

### Encode and decode a local video

In [ ]:
!wget http://www.hpca.ual.es/~vruiz/videos/coastguard_352x288x30x420x300.avi

In [ ]:
# Encodes 80 frames (gop_size=16 * num_gops=5)
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("../tmp/encoded.mp4")

### Default encoding and decoding

In [ ]:
!rm /tmp/encoded* /tmp/decoded* 2>/dev/null; echo "Cleaned /tmp files"

In [ ]:
!python3 ../src/MCTF.py encode -o "your_video"

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Using hierarchical B-frame structure

In [ ]:
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5 --hierarchical

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Using fast motion estimation

In [ ]:
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5 --fast

In [ ]:
!python3 ../src/MCTF.py decode

In [ ]:
Video("/tmp/decoded.mp4")

### Different quantization steps

In [ ]:
# Lower QSS = better quality
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5 -q 16 --fast

In [ ]:
!python3 ../src/MCTF.py decode -q 16

In [ ]:
Video("/tmp/decoded.mp4")

In [ ]:
# Higher QSS = more compression
!python3 ../src/MCTF.py encode -o coastguard_352x288x30x420x300.avi --gop_size 16 --num_gops 5 -q 64 --fast

In [ ]:
!python3 ../src/MCTF.py decode -q 64

In [ ]:
Video("/tmp/decoded.mp4")

### Hierarchical vs Non-Hierarchical B-frames Comparison

This comparison demonstrates the compression efficiency difference between simple (sequential) and hierarchical B-frame structures.

In [23]:
# Clean up previous encodings
!rm -f /tmp/encoded_* /tmp/decoded_*

import re
import subprocess

In [ ]:
%%time
# Encode WITHOUT hierarchical B-frames
print("=" * 70)
print("ENCODING: Simple (Non-Hierarchical) B-frames")
print("=" * 70)

result = subprocess.run(
    ['python3', '../src/MCTF.py', 'encode', 
     '-o', 'coastguard_352x288x30x420x300.avi',
     '--gop_size', '16',
     '--num_gops', '5',
     '-q', '32',
     '--fast',
     '-O', '/tmp/encoded_simple'],
    capture_output=True,
    text=True
)

print(f"result stdout {result.stdout}")
print(f"result stderr {result.stderr}")

# Extract BPP from output
bpp_line = [l for l in result.stderr.split('\n') if 'Bits Per Pixel (BPP):' in l][0]
print(f"Found line: {bpp_line}")  # Debug

bpp_simple = float(bpp_line.split(':')[-1].strip())  

print(f"\n✓ Hierarchical B-frames BPP: {bpp_simple:.6f}")

In [ ]:
%%time
# Encode WITH hierarchical B-frames
print("=" * 70)
print("ENCODING: Hierarchical B-frames")
print("=" * 70)

result = subprocess.run(
    ['python3', '../src/MCTF.py', 'encode', 
     '-o', 'coastguard_352x288x30x420x300.avi',
     '--gop_size', '16',
     '--num_gops', '5',
     '-q', '32',
     '--fast',
     '--hierarchical',
     '-O', '/tmp/encoded_hierarchical'],
    capture_output=True,
    text=True
)

print(f"result stdout {result.stdout}")
print(f"result stderr {result.stderr}")

# Extract BPP from output
bpp_line = [l for l in result.stderr.split('\n') if 'Bits Per Pixel (BPP):' in l][0]
print(f"Found line: {bpp_line}")  
bpp_hierarchical = float(bpp_line.split(':')[-1].strip())

print(f"\n✓ Hierarchical B-frames BPP: {bpp_hierarchical:.6f}")

In [ ]:
pip install pandas

In [ ]:
# Display comparison table
import pandas as pd

if bpp_simple and bpp_hierarchical:
    improvement = ((bpp_simple - bpp_hierarchical) / bpp_simple) * 100
    
    comparison_data = {
        'Mode': ['Simple (Non-Hierarchical)', 'Hierarchical'],
        'BPP': [f'{bpp_simple:.6f}', f'{bpp_hierarchical:.6f}'],
        'Bytes/Frame (estimated)': [
            f'{(bpp_simple * 352 * 288 / 8):.2f}',
            f'{(bpp_hierarchical * 352 * 288 / 8):.2f}'
        ]
    }
    
    df = pd.DataFrame(comparison_data)
    display(df)
    
    print(f"\n{'=' * 70}")
    print(f"COMPRESSION COMPARISON RESULTS")
    print(f"{'=' * 70}")
    print(f"Simple B-frames:        {bpp_simple:.6f} bpp")
    print(f"Hierarchical B-frames:  {bpp_hierarchical:.6f} bpp")
    print(f"{'=' * 70}")
    
    if improvement > 0:
        print(f"✓ Hierarchical is {improvement:.2f}% BETTER (lower BPP)")
        print(f"  → Savings: {bpp_simple - bpp_hierarchical:.6f} bpp")
    else:
        print(f"✗ Hierarchical is {abs(improvement):.2f}% WORSE (higher BPP)")
    
    print(f"\nWhy hierarchical is better:")
    print("  • B-frames reference temporally closer frames")
    print("  • Smaller motion vectors (less to encode)")
    print("  • More accurate prediction (smaller residuals)")
    print("  • Better rate-distortion performance overall")
else:
    print("⚠ Could not extract BPP values. Please check the encoding output.")

In [ ]:
# Decode hierarchical version for playback
!python3 ../src/MCTF.py decode -i /tmp/encoded_hierarchical -O /tmp/decoded_hierarchical

In [ ]:
Video("/tmp/decoded_hierarchical.mp4", width=640)